In [ ]:
import pandas as pd

In [ ]:
excel_table_path = "data/input/tables/table.xlsx"

tables = pd.read_excel(excel_table_path, sheet_name="valid_list")
tables = tables.loc[tables["VALID"].isin(["O", "o"])]
del tables["VALID"]
tables

In [ ]:
def valid_type_mapper(before_schema_type: str, after_schema_type: str) -> bool:
    result = False

    m = {
        "VARCHAR2": ["VARCHAR"],
        "CHAR": ["VARCHAR"],
        "DATE": ["DATETIME"],
        "NUMBER": ["INT", "BIGINT", "DECIMAL"],
    }

    try:
        result = after_schema_type in m[before_schema_type]
    except KeyError:
        # 원본 데이터 타입이 잘못 적힌 경우, 확인이 필요하기 떄문에 False로 반환
        pass

    return result


def valid_number_mapper(num_type: str, length: str) -> bool:
    result = False

    if num_type == "DECIMAL":
        # 숫자길이가 20자리 이상인 경우
        try:
            result = int(length) > 19
        except ValueError:
            # 정밀도 표현 때문에 콜론이 들어가 있어서 ValueError 발생
            pass

        try:
            if "," in length:
                result = True
        except TypeError:
            pass

    elif num_type == "INT":
        try:
            result = int(length) < 10
        except ValueError:
            # 잘못 작성한 경우
            pass

    elif num_type == "BIGINT":
        try:
            result = 9 < int(length) < 20
        except ValueError:
            # 잘못 작성한 경우
            pass

    return result

In [ ]:
db_name = ""
schema_name = ""
excel_define_path = "data/input/defines/define_latest.xlsx"


for table_name in tables.loc[
    tables["DB"].str.contains(db_name) & tables["SCHEMA"].str.contains(schema_name)
]["TABLE"]:
    df_before = pd.read_excel(
        excel_define_path,
        sheet_name=table_name,
        skiprows=11,
        skipfooter=5,
        usecols="A:I",
    )

    df_after = pd.read_excel(
        excel_define_path,
        sheet_name=table_name,
        skiprows=11,
        skipfooter=5,
        usecols="J:T",
    )
    df_after = df_after.loc[df_after["SEQ.1"].notnull()]

    try:
        df = df_after.merge(
            right=df_before, how="inner", left_on="Mapping SEQ", right_on="SEQ"
        )
    except Exception as E:
        print(f"확인 필요: {table_name}")
        print(E)

    if df is None:
        print(f"확인 필요: {table_name}")
        continue

    #
    df_val = df.loc[df["필드.1"] != df["필드"]]

    if not df_val.empty:
        for i, row in df_val.iterrows():
            print(i, row)

    df_val = df.loc[
        ~df.apply(
            lambda row: valid_type_mapper(
                before_schema_type=row["타입"], after_schema_type=row["타입.1"]
            ),
            axis=1,
        )
    ]

    if not df_val.empty:
        for i, row in df_val.iterrows():
            print(i, row)

    df_val = df.loc[
        ~df.apply(
            lambda row: valid_number_mapper(
                num_type=row["타입.1"], length=row["길이.1"]
            ),
            axis=1,
        )
    ].loc[~df["타입.1"].isin(["VARCHAR", "DATETIME"])]

    if not df_val.empty:
        for i, row in df_val.iterrows():
            print(i, row)